In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.utils import resample
from collections import Counter

In [2]:
cd tripadvisor/reviews/csv

/home/joanna_huang/W266-Final-Project/tripadvisor/reviews/csv


In [3]:
filepath = ['Chicago_Illinois_1.csv','Chicago_Illinois_2.csv', 'Las_Vegas_Nevada_1.csv', 'Las_Vegas_Nevada_2.csv',
'Las_Vegas_Nevada_3.csv','Las_Vegas_Nevada_4.csv','Las_Vegas_Nevada_5.csv', 'Las_Vegas_Nevada_6.csv',
'Las_Vegas_Nevada_7.csv', 'New_York_City_New_York_1.csv', 'New_York_City_New_York_2.csv', 
'New_York_City_New_York_3.csv', 'New_York_City_New_York_4.csv', 'New_York_City_New_York_5.csv',
'New_York_City_New_York_6.csv', 'New_York_City_New_York_7.csv', 'San_Francisco_California_1.csv',
'San_Francisco_California_2.csv']

data = pd.DataFrame()
for path in filepath:
    df = pd.read_csv(path)
    df['city'] = path.split('_')[0]
    data = data.append(df)
data.shape #(968772, 19)
data = data.reset_index(drop=True)
del data['neighborhood'] #remove column given 80+% null
data['review_body'] = [x.lower() for x in data['review_body']]
data['review_body'] = [x.split('.') for x in data['review_body']]

/home/joanna_huang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cd

/home/joanna_huang


In [5]:
cd W266-Final-Project/

/home/joanna_huang/W266-Final-Project


In [6]:
df2 = pd.read_csv('df2.all')
df2['Rating'] = df2['Rating'].map({50: 3, 40: 2, 30: 1, 20: 0, 10: 0})
df2['city'] = data['city']
df2['city'] = df2['city'].map({'Chicago': 1, 'New': 2, 'San': 3, 'Las': 4})
df2['affect'] = df2['affect']*0.01
df2['posemo'] = df2['posemo']*0.01
df2['negemo'] = df2['negemo']*0.01
df2['anx'] = df2['anx']*0.01
df2['anger'] = df2['anger']*0.01
df2['sad'] = df2['sad']*0.01
df2['cogproc'] = df2['cogproc']*0.01
df2['insight'] = df2['insight']*0.01
df2['cause'] = df2['cause']*0.01
df2['discrep'] = df2['discrep']*0.01
df2['tentat'] = df2['tentat']*0.01
df2['certain'] = df2['certain']*0.01
df2['differ'] = df2['differ']*0.01

In [12]:
# Calculate word percentages
df2['fp_perc'] = df2['First Person']/df2['Word_Count']
df2['tp_perc'] = df2['Third Person']/df2['Word_Count']
df2['fear_perc'] = df2['Fear']/df2['Word_Count']
df2['trust_perc'] = df2['Trust']/df2['Word_Count']
df2['sad_perc'] = df2['Sad']/df2['Word_Count']
df2['affect_perc'] = df2['affect']/df2['Word_Count']
df2['cogproc_perc'] = df2['cogproc']/df2['Word_Count']
df2['negemo_perc'] = df2['negemo']/df2['Word_Count']
df2['posemo_perc'] = df2['posemo']/df2['Word_Count']
df2['past_perc'] = df2['Past Tense']/df2['Word_Count']
df2['narr_perc'] = df2['Narrative Seq']/df2['Word_Count']
df2['adj_perc'] = df2['Adjectives']/df2['Word_Count']
df2['joy_perc'] = df2['Joy']/df2['Word_Count']
df2['disgust_perc'] = df2['Disgust']/df2['Word_Count']
df2['sup_perc'] = df2['Superlatives']/df2['Word_Count']
df2['nouns_perc'] = df2['Nouns']/df2['Word_Count']
df2['comp_perc'] = df2['Comparatives']/df2['Word_Count']

In [68]:
y = df2['Rating']
X = df2[['Word_Count','Helpful', 'city', 'class', 'activity_level',
         'tp_perc', 'fp_perc', 'negemo_perc', 'affect_perc','disgust_perc', 'joy_perc', 'sad_perc',
         'narr_perc', 'past_perc','spatial', 'temporal','Polarity2', 'cogproc_perc','comp_perc']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [70]:
# Multinomial Logistic Regression
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(X_train, y_train)
y_pred_mul = mul_lr.predict(X_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred_mul)
print(confusion_matrix)
print(metrics.classification_report(y_test, y_pred_mul))
print(metrics.accuracy_score(y_test, y_pred_mul))

/home/joanna_huang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/joanna_huang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


[[ 22173   4359   2954   3651]
 [  8959   6592  11818  11658]
 [  3297   4467  23841  53692]
 [  1900   2117  16512 131081]]
             precision    recall  f1-score   support

          0       0.61      0.67      0.64     33137
          1       0.38      0.17      0.23     39027
          2       0.43      0.28      0.34     85297
          3       0.66      0.86      0.75    151610

avg / total       0.55      0.59      0.56    309071

0.5943197517722465


In [ ]:
0.5943 with comparatives


In [69]:
import statsmodels.api as st
mdl = st.MNLogit(y_train, X_train)
mdl_fit = mdl.fit()
print(mdl_fit.summary())

Optimization terminated successfully.
         Current function value: 0.909009
         Iterations 9
                          MNLogit Regression Results                          
Dep. Variable:                 Rating   No. Observations:               721163
Model:                        MNLogit   Df Residuals:                   721106
Method:                           MLE   Df Model:                           54
Date:                Wed, 05 Dec 2018   Pseudo R-squ.:                  0.2459
Time:                        08:13:56   Log-Likelihood:            -6.5554e+05
converged:                       True   LL-Null:                   -8.6928e+05
                                        LLR p-value:                     0.000
      Rating=1       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Word_Count        -0.0012   3.86e-05    -30.481      0.000      -0.001      -0.001
Helpful          

In [67]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_test.values, i) for i in range(X_test.shape[1])]
vif["features"] = X_test.columns
vif

,VIF Factor,features
0,2.601770,Word_Count
1,1.163950,Helpful
2,7.233473,city
3,20.242719,class
4,9.261926,activity_level
5,2.164241,tp_perc
6,1.897815,fp_perc
7,1.326495,negemo_perc
8,2.772768,affect_perc
9,1.580320,disgust_perc


In [57]:
results_df = X_test
results_df['y_test']= y_test
results_df['y_pred']=y_pred_mul
InCorrLow_0 = results_df[(results_df['y_test']!=0) & (results_df['y_pred']==0)]
InCorrLow_1 = results_df[(results_df['y_test']!=1) & (results_df['y_pred']==1)]
InCorrHigh_2 = results_df[(results_df['y_test']!=2) & (results_df['y_pred']==2)]
InCorrHigh_3 = results_df[(results_df['y_test']!=3) & (results_df['y_pred']==3)]
CorrLow_0 = results_df[(results_df['y_test']==0) & (results_df['y_pred']==0)]
CorrLow_1 = results_df[(results_df['y_test']==1) & (results_df['y_pred']==1)]
CorrHigh_2 = results_df[(results_df['y_test']==2) & (results_df['y_pred']==2)]
CorrHigh_3 = results_df[(results_df['y_test']==3) & (results_df['y_pred']==3)]


/home/joanna_huang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joanna_huang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
InCorrLow_0['corr_fp'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['fp_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_negemo'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['negemo_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_affect'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['affect_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_disgust'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['disgust_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_joy'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['joy_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_sad'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['sad_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_narr'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['narr_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_past'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['past_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_Polarity'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['Polarity2']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_cogproc'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['cogproc_perc']),2) \
                          for item in InCorrLow_0.index.values]
InCorrLow_0['corr_comp'] = [round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['comp_perc']),2) \
                          for item in InCorrLow_0.index.values]

In [59]:
len(InCorrLow_0)

14156

In [60]:
for item in InCorrLow_0.sample(n=100).index.values:
    print('Predicted: ' + str(InCorrLow_0.loc[item]['y_pred']) + ' Actual: ' + str(InCorrLow_0.loc[item]['y_test']))
    print('Third Person %: ' + str(round(InCorrLow_0.loc[item]['tp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['tp_perc']),2)))
    print('First Person %: ' + str(round(InCorrLow_0.loc[item]['fp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['fp_perc']),2)))
    print('Negemo %: ' + str(round(InCorrLow_0.loc[item]['negemo_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['negemo_perc']),2)))
    print('Affect %: ' + str(round(InCorrLow_0.loc[item]['affect_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['affect_perc']),2)))
    print('Disgust %: ' + str(round(InCorrLow_0.loc[item]['disgust_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['disgust_perc']),2)))
    print('Joy %: ' + str(round(InCorrLow_0.loc[item]['joy_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['joy_perc']),2)))
    print('Sad %: ' + str(round(InCorrLow_0.loc[item]['sad_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['sad_perc']),2)))
    print('Narr %: ' + str(round(InCorrLow_0.loc[item]['narr_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['narr_perc']),2)))
    print('Past %: ' + str(round(InCorrLow_0.loc[item]['past_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['past_perc']),2)))
    print('Polarity2 %: ' + str(round(InCorrLow_0.loc[item]['Polarity2'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['Polarity2']),2)))
    print('Cogproc %: ' + str(round(InCorrLow_0.loc[item]['cogproc_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['cogproc_perc']),2)))
    print('Comp %: ' + str(round(InCorrLow_0.loc[item]['comp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_0.loc[item]['y_test']]['comp_perc']),2)))
    print('\n')

Predicted: 0.0 Actual: 1.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.01 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.06 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.02 Correct average: 0.02
Sad %: 0.0 Correct average: 0.01
Narr %: 0.03 Correct average: 0.01
Past %: 0.1 Correct average: 0.08
Polarity2 %: 0.1 Correct average: 0.13
Cogproc %: 0.09 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 3.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.0 Correct average: 0.04
Sad %: 0.05 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.02 Correct average: 0.07
Polarity2 %: -0.07 Correct average: 0.33
Cogproc %: 0.11 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.0 Correct av

Disgust %: 0.01 Correct average: 0.01
Joy %: 0.0 Correct average: 0.02
Sad %: 0.03 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.13 Correct average: 0.08
Polarity2 %: 0.11 Correct average: 0.13
Cogproc %: 0.14 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 2.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.01 Correct average: 0.08
Disgust %: 0.01 Correct average: 0.0
Joy %: 0.0 Correct average: 0.03
Sad %: 0.01 Correct average: 0.01
Narr %: 0.03 Correct average: 0.01
Past %: 0.07 Correct average: 0.07
Polarity2 %: 0.07 Correct average: 0.25
Cogproc %: 0.1 Correct average: 0.09
Comp %: 0.01 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.07 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.03 Correct average: 0.06
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.03 Correc

Polarity2 %: 0.09 Correct average: 0.13
Cogproc %: 0.12 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.0 Correct average: 0.06
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.02 Correct average: 0.02
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.12 Correct average: 0.08
Polarity2 %: 0.03 Correct average: 0.13
Cogproc %: 0.06 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.03 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.05 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.02 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.13 Correct average: 0.08
Polarity2 %: 0.04 Correct average: 0.13
Cogproc %: 0.09 

First Person %: 0.03 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.03 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.02 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.08 Correct average: 0.08
Polarity2 %: 0.03 Correct average: 0.13
Cogproc %: 0.07 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.01 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.02 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.01 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.12 Correct average: 0.08
Polarity2 %: 0.1 Correct average: 0.13
Cogproc %: 0.09 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 3.0
Third Person %: 0.04 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 

Joy %: 0.0 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.1 Correct average: 0.07
Polarity2 %: 0.08 Correct average: 0.25
Cogproc %: 0.11 Correct average: 0.09
Comp %: 0.02 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.03 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.05 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.01 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.08 Correct average: 0.08
Polarity2 %: 0.09 Correct average: 0.13
Cogproc %: 0.12 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 3.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.02 Correct average: 0.04
Sad %: 0.02 Correct av

Cogproc %: 0.2 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.04 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.03 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.07 Correct average: 0.08
Polarity2 %: 0.13 Correct average: 0.13
Cogproc %: 0.16 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 0.0 Actual: 1.0
Third Person %: 0.04 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.05 Correct average: 0.01
Affect %: 0.06 Correct average: 0.06
Disgust %: 0.03 Correct average: 0.01
Joy %: 0.03 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.0 Correct average: 0.08
Polarity2 %: -0.08 Correct average: 0.13
Cogproc %: 0.1 Correct average: 0.1
Comp %: 0.0 Corre

In [61]:
for item in InCorrLow_1.sample(n=100).index.values:
    print('Predicted: ' + str(InCorrLow_1.loc[item]['y_pred']) + ' Actual: ' + str(InCorrLow_1.loc[item]['y_test']))
    print('Third Person %: ' + str(round(InCorrLow_1.loc[item]['tp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['tp_perc']),2)))
    print('First Person %: ' + str(round(InCorrLow_1.loc[item]['fp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['fp_perc']),2)))
    print('Negemo %: ' + str(round(InCorrLow_1.loc[item]['negemo_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['negemo_perc']),2)))
    print('Affect %: ' + str(round(InCorrLow_1.loc[item]['affect_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['affect_perc']),2)))
    print('Disgust %: ' + str(round(InCorrLow_1.loc[item]['disgust_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['disgust_perc']),2)))
    print('Joy %: ' + str(round(InCorrLow_1.loc[item]['joy_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['joy_perc']),2)))
    print('Sad %: ' + str(round(InCorrLow_1.loc[item]['sad_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['sad_perc']),2)))
    print('Narr %: ' + str(round(InCorrLow_1.loc[item]['narr_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['narr_perc']),2)))
    print('Past %: ' + str(round(InCorrLow_1.loc[item]['past_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['past_perc']),2)))
    print('Polarity2 %: ' + str(round(InCorrLow_1.loc[item]['Polarity2'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['Polarity2']),2)))
    print('Cogproc %: ' + str(round(InCorrLow_1.loc[item]['cogproc_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['cogproc_perc']),2)))
    print('Comp %: ' + str(round(InCorrLow_1.loc[item]['comp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrLow_1.loc[item]['y_test']]['comp_perc']),2)))
    print('\n')

Predicted: 1.0 Actual: 0.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.01 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.03
Affect %: 0.01 Correct average: 0.05
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.01 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.16 Correct average: 0.09
Polarity2 %: 0.11 Correct average: 0.01
Cogproc %: 0.06 Correct average: 0.1
Comp %: 0.01 Correct average: 0.0


Predicted: 1.0 Actual: 0.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.03
Affect %: 0.03 Correct average: 0.05
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.03 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.03 Correct average: 0.01
Past %: 0.11 Correct average: 0.09
Polarity2 %: 0.08 Correct average: 0.01
Cogproc %: 0.15 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.02 Corre

Joy %: 0.0 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.12 Correct average: 0.09
Polarity2 %: 0.02 Correct average: 0.01
Cogproc %: 0.16 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.04 Correct average: 0.01
Affect %: 0.12 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.04 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.16 Correct average: 0.07
Polarity2 %: 0.12 Correct average: 0.25
Cogproc %: 0.1 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 3.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.01 Correct average: 0.0
Joy %: 0.02 Correct average: 0.04
Sad %: 0.01 Correct average

Comp %: 0.02 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.09 Correct average: 0.08
Disgust %: 0.02 Correct average: 0.0
Joy %: 0.07 Correct average: 0.03
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.02 Correct average: 0.07
Polarity2 %: 0.08 Correct average: 0.25
Cogproc %: 0.18 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.01 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.07 Correct average: 0.08
Disgust %: 0.01 Correct average: 0.0
Joy %: 0.01 Correct average: 0.03
Sad %: 0.01 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.06 Correct average: 0.07
Polarity2 %: 0.07 Correct average: 0.25
Cogproc %: 0.16 Correct average: 0.09
Comp %: 0.02 Correct average: 0.0


Predicted: 1.0 

Affect %: 0.11 Correct average: 0.08
Disgust %: 0.03 Correct average: 0.0
Joy %: 0.04 Correct average: 0.03
Sad %: 0.03 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.1 Correct average: 0.07
Polarity2 %: 0.11 Correct average: 0.25
Cogproc %: 0.09 Correct average: 0.09
Comp %: 0.01 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.05 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.03 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.0 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.05 Correct average: 0.07
Polarity2 %: 0.04 Correct average: 0.25
Cogproc %: 0.12 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 3.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.03 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.01 Correc

Past %: 0.08 Correct average: 0.07
Polarity2 %: 0.08 Correct average: 0.33
Cogproc %: 0.12 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 3.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.0 Correct average: 0.04
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.05 Correct average: 0.07
Polarity2 %: 0.07 Correct average: 0.33
Cogproc %: 0.09 Correct average: 0.08
Comp %: 0.02 Correct average: 0.0


Predicted: 1.0 Actual: 3.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.0
Affect %: 0.07 Correct average: 0.1
Disgust %: 0.01 Correct average: 0.0
Joy %: 0.01 Correct average: 0.04
Sad %: 0.01 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.11 Correct average: 0.07
Polarity2 %: 0.12 Correct a

First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.03
Affect %: 0.03 Correct average: 0.05
Disgust %: 0.02 Correct average: 0.01
Joy %: 0.05 Correct average: 0.02
Sad %: 0.04 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.04 Correct average: 0.09
Polarity2 %: 0.01 Correct average: 0.01
Cogproc %: 0.18 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 0.0
Third Person %: 0.04 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.03
Affect %: 0.05 Correct average: 0.05
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.03 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.04 Correct average: 0.09
Polarity2 %: 0.08 Correct average: 0.01
Cogproc %: 0.1 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 1.0 Actual: 2.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.

In [62]:
for item in InCorrHigh_2.sample(n=100).index.values:
    print('Predicted: ' + str(InCorrHigh_2.loc[item]['y_pred']) + ' Actual: ' + str(InCorrHigh_2.loc[item]['y_test']))
    print('Third Person %: ' + str(round(InCorrHigh_2.loc[item]['tp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['tp_perc']),2)))
    print('First Person %: ' + str(round(InCorrHigh_2.loc[item]['fp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['fp_perc']),2)))
    print('Negemo %: ' + str(round(InCorrHigh_2.loc[item]['negemo_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['negemo_perc']),2)))
    print('Affect %: ' + str(round(InCorrHigh_2.loc[item]['affect_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['affect_perc']),2)))
    print('Disgust %: ' + str(round(InCorrHigh_2.loc[item]['disgust_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['disgust_perc']),2)))
    print('Joy %: ' + str(round(InCorrHigh_2.loc[item]['joy_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['joy_perc']),2)))
    print('Sad %: ' + str(round(InCorrHigh_2.loc[item]['sad_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['sad_perc']),2)))
    print('Narr %: ' + str(round(InCorrHigh_2.loc[item]['narr_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['narr_perc']),2)))
    print('Past %: ' + str(round(InCorrHigh_2.loc[item]['past_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['past_perc']),2)))
    print('Polarity2 %: ' + str(round(InCorrHigh_2.loc[item]['Polarity2'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['Polarity2']),2)))
    print('Cogproc %: ' + str(round(InCorrHigh_2.loc[item]['cogproc_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['cogproc_perc']),2)))
    print('Comp %: ' + str(round(InCorrHigh_2.loc[item]['comp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_2.loc[item]['y_test']]['comp_perc']),2)))
    print('\n')

Predicted: 2.0 Actual: 3.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.0
Affect %: 0.12 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.04 Correct average: 0.04
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.0 Correct average: 0.07
Polarity2 %: 0.2 Correct average: 0.33
Cogproc %: 0.12 Correct average: 0.08
Comp %: 0.02 Correct average: 0.0


Predicted: 2.0 Actual: 1.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.06 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.04 Correct average: 0.02
Sad %: 0.04 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.03 Correct average: 0.08
Polarity2 %: 0.15 Correct average: 0.13
Cogproc %: 0.11 Correct average: 0.1
Comp %: 0.01 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.03 Correct a

Joy %: 0.03 Correct average: 0.04
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.08 Correct average: 0.07
Polarity2 %: 0.16 Correct average: 0.33
Cogproc %: 0.1 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.04 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.0
Affect %: 0.06 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.02 Correct average: 0.04
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.09 Correct average: 0.07
Polarity2 %: 0.3 Correct average: 0.33
Cogproc %: 0.1 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 1.0
Third Person %: 0.06 Correct average: 0.02
First Person %: 0.01 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.01
Affect %: 0.09 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.02 Correct average: 0.02
Sad %: 0.01 Correct a

Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.0
Affect %: 0.07 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.03 Correct average: 0.04
Sad %: 0.02 Correct average: 0.01
Narr %: 0.02 Correct average: 0.01
Past %: 0.1 Correct average: 0.07
Polarity2 %: 0.16 Correct average: 0.33
Cogproc %: 0.12 Correct average: 0.08
Comp %: 0.02 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.0
Affect %: 0.09 Correct average: 0.1
Disgust %: 0.01 Correct average: 0.0
Joy %: 0.09 Correct average: 0.04
Sad %: 0.03 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.01 Correct average: 0.07
Polarity2 %: 0.24 Correct average: 0.33
Cogproc %: 0.09 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 

Disgust %: 0.0 Correct average: 0.01
Joy %: 0.0 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.14 Correct average: 0.08
Polarity2 %: 0.12 Correct average: 0.13
Cogproc %: 0.06 Correct average: 0.1
Comp %: 0.01 Correct average: 0.0


Predicted: 2.0 Actual: 0.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.03 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.03
Affect %: 0.05 Correct average: 0.05
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.04 Correct average: 0.02
Sad %: 0.03 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.07 Correct average: 0.09
Polarity2 %: 0.15 Correct average: 0.01
Cogproc %: 0.06 Correct average: 0.1
Comp %: 0.01 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.06 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.0
Affect %: 0.1 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.03 Correc

Cogproc %: 0.21 Correct average: 0.1
Comp %: 0.06 Correct average: 0.0


Predicted: 2.0 Actual: 1.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.05 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.06 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.05 Correct average: 0.08
Polarity2 %: 0.09 Correct average: 0.13
Cogproc %: 0.14 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.13 Correct average: 0.06
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.07 Correct average: 0.02
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.0 Correct average: 0.08
Polarity2 %: 0.36 Correct average: 0.13
Cogproc %: 0.08 Correct average: 0.1
Comp %: 0.0 Correct a

Negemo %: 0.01 Correct average: 0.0
Affect %: 0.04 Correct average: 0.1
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.01 Correct average: 0.04
Sad %: 0.01 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.08 Correct average: 0.07
Polarity2 %: 0.16 Correct average: 0.33
Cogproc %: 0.11 Correct average: 0.08
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 0.0
Third Person %: 0.03 Correct average: 0.02
First Person %: 0.04 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.03
Affect %: 0.04 Correct average: 0.05
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.01 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.1 Correct average: 0.09
Polarity2 %: 0.06 Correct average: 0.01
Cogproc %: 0.09 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 2.0 Actual: 3.0
Third Person %: 0.01 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.0
Affect %: 0.04 Correc

In [64]:
for item in InCorrHigh_3.sample(n=100).index.values:
    print('Predicted: ' + str(InCorrHigh_3.loc[item]['y_pred']) + ' Actual: ' + str(InCorrHigh_3.loc[item]['y_test']))
    print('Third Person %: ' + str(round(InCorrHigh_3.loc[item]['tp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['tp_perc']),2)))
    print('First Person %: ' + str(round(InCorrHigh_3.loc[item]['fp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['fp_perc']),2)))
    print('Negemo %: ' + str(round(InCorrHigh_3.loc[item]['negemo_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['negemo_perc']),2)))
    print('Affect %: ' + str(round(InCorrHigh_3.loc[item]['affect_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['affect_perc']),2)))
    print('Disgust %: ' + str(round(InCorrHigh_3.loc[item]['disgust_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['disgust_perc']),2)))
    print('Joy %: ' + str(round(InCorrHigh_3.loc[item]['joy_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['joy_perc']),2)))
    print('Sad %: ' + str(round(InCorrHigh_3.loc[item]['sad_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['sad_perc']),2)))
    print('Narr %: ' + str(round(InCorrHigh_3.loc[item]['narr_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['narr_perc']),2)))
    print('Past %: ' + str(round(InCorrHigh_3.loc[item]['past_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['past_perc']),2)))
    print('Polarity2 %: ' + str(round(InCorrHigh_3.loc[item]['Polarity2'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['Polarity2']),2)))
    print('Cogproc %: ' + str(round(InCorrHigh_3.loc[item]['cogproc_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['cogproc_perc']),2)))
    print('Comp %: ' + str(round(InCorrHigh_3.loc[item]['comp_perc'],2)) + ' Correct average: ' + 
          str(round(np.mean(df2[df2['Rating']==InCorrHigh_3.loc[item]['y_test']]['comp_perc']),2)))
    print('\n')

Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.21 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.06 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.12 Correct average: 0.07
Polarity2 %: 0.58 Correct average: 0.25
Cogproc %: 0.05 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 0.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.02 Correct average: 0.02
Negemo %: 0.01 Correct average: 0.03
Affect %: 0.04 Correct average: 0.05
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.03 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.11 Correct average: 0.09
Polarity2 %: 0.15 Correct average: 0.01
Cogproc %: 0.15 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.03 Correct a

Disgust %: 0.0 Correct average: 0.0
Joy %: 0.03 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.02 Correct average: 0.07
Polarity2 %: 0.3 Correct average: 0.25
Cogproc %: 0.12 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.1 Correct average: 0.06
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.01 Correct average: 0.02
Sad %: 0.01 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.06 Correct average: 0.08
Polarity2 %: 0.11 Correct average: 0.13
Cogproc %: 0.08 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.04 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.04 Correct average: 0.06
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.02 Correct a

Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.03 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.07 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.05 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.05 Correct average: 0.07
Polarity2 %: 0.33 Correct average: 0.25
Cogproc %: 0.02 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.3 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.11 Correct average: 0.03
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.0 Correct average: 0.07
Polarity2 %: 0.43 Correct average: 0.25
Cogproc %: 0.09 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 1.0


Disgust %: 0.0 Correct average: 0.0
Joy %: 0.15 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.0 Correct average: 0.07
Polarity2 %: 0.45 Correct average: 0.25
Cogproc %: 0.03 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 1.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.01
Affect %: 0.09 Correct average: 0.06
Disgust %: 0.01 Correct average: 0.01
Joy %: 0.05 Correct average: 0.02
Sad %: 0.02 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.07 Correct average: 0.08
Polarity2 %: 0.22 Correct average: 0.13
Cogproc %: 0.07 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 0.0
Third Person %: 0.02 Correct average: 0.02
First Person %: 0.04 Correct average: 0.02
Negemo %: 0.02 Correct average: 0.03
Affect %: 0.12 Correct average: 0.05
Disgust %: 0.0 Correct average: 0.01
Joy %: 0.08 Correct

Polarity2 %: 0.04 Correct average: 0.01
Cogproc %: 0.14 Correct average: 0.1
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.16 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.0 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.02 Correct average: 0.07
Polarity2 %: 0.64 Correct average: 0.25
Cogproc %: 0.06 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.11 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.03 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.01 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.01 Correct average: 0.01
Past %: 0.06 Correct average: 0.07
Polarity2 %: 0.04 Correct average: 0.25
Cogproc %: 0.07 Correc

First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.08 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.05 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.03 Correct average: 0.07
Polarity2 %: 0.3 Correct average: 0.25
Cogproc %: 0.11 Correct average: 0.09
Comp %: 0.0 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct average: 0.01
Affect %: 0.0 Correct average: 0.08
Disgust %: 0.0 Correct average: 0.0
Joy %: 0.01 Correct average: 0.03
Sad %: 0.0 Correct average: 0.01
Narr %: 0.0 Correct average: 0.01
Past %: 0.05 Correct average: 0.07
Polarity2 %: 0.26 Correct average: 0.25
Cogproc %: 0.0 Correct average: 0.09
Comp %: 0.01 Correct average: 0.0


Predicted: 3.0 Actual: 2.0
Third Person %: 0.0 Correct average: 0.02
First Person %: 0.0 Correct average: 0.02
Negemo %: 0.0 Correct 